<a href="https://colab.research.google.com/github/biplobsd/pecResultDownloadSort/blob/master/pecResultDownloadSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# <img src='http://dperesult.teletalk.com.bd/images/index__02.jpg' height="25" alt="Peerflix-server" /> __From dpe__

In [0]:
#email biplobsd11@gmail.com
#@markdown <h3>⬅️ Click Here to START DOWNLOADING.</h3>
#@markdown <br><center><img src='http://dperesult.teletalk.com.bd/images/index__01.jpg' height="200" alt="PEC"/></center>
#@markdown <center><h3>PEC<br />Batch download marks sheet</h3></center><br>
PASSING_YEAR = 2019 #@param ["2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010"] {type:"raw"}
#@markdown Student ID range  :
START =  1120193080500071#@param {type:"number"}
END =  1120193080500181#@param {type:"number"}
import requests, pathlib, os, re
from bs4 import BeautifulSoup
from threading import Thread
from IPython.display import clear_output
headers = {
    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.46 Safari/537.36',
}

pathlib.Path('dpe').mkdir(mode=0o777, exist_ok=True)
url = "http://dperesult.teletalk.com.bd/dpe_result.php"

login_data = {
  "passing_year": PASSING_YEAR,
  "yes": "YES",
  "submit": "Submit"
}

selTable = {
    "align": "center",
    "width": "550",
    "border": "0",
    "cellpadding":"3",
    "cellspacing": "1",
    "class": "black12bold"
}

def buildRange(START, END, numsplits):
    value = (END+1) - START
    lst = []
    for i in range(numsplits):
        if i == 0:
            lst.append([START, START + int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))])
        else:
            lst.append([START + int(round(1 + i * value/(numsplits*1.0),0)), START + int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))])
    return lst

splitRange = buildRange(START, END, 2)
def downloader(START, END):
    with requests.Session() as s:
        for roll in range(START, END):
          if not os.path.isfile(f'dpe/{roll}.html'):
            login_data['roll'] = roll
            for do in range(10):
              try:
                content = s.post(url, data=login_data, headers=headers).text
                soup = soup = BeautifulSoup(content, 'html5lib').find("table", attrs=selTable).text
                sid = int(re.findall(r'Student\sID\n\t{2}:\n\t{2}\d{1,16}', soup)[0].split('\n')[2])
                if sid == roll:
                  with open(f'dpe/{roll}.html', 'w') as f:
                      f.write(content)
                  print(f"Done {roll}")
                break
              except ValueError:
                  print(f"id {roll} Retrying... {do}")
                  continue
          else:
                print(f"Done {roll}")
for ck in range(3):
  downloaders = [
          Thread(
              target=downloader, 
              args=(START, END),
          )
          for START,END in splitRange
          ]
  for th in downloaders:
      th.start()
  for th in downloaders:
      th.join()

print("All file in /content/dpe")

In [0]:
#email biplobsd11@gmail.com
#@markdown <h3>⬅️ Click Here to START SORTING.</h3>
#@markdown <br><center><img src='http://dperesult.teletalk.com.bd/images/index__01.jpg' height="200" alt="PEC"/></center>
#@markdown <center><h3>PEC<br />Sort marks sheet</h3></center><br>
#@markdown Student ID range  :
START =  1120193080500071#@param {type:"number"}
END =  1120193080500181#@param {type:"number"}
#@markdown Subject priority :
Total_Marks = "Total Marks" #@param ["Total Marks","Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
FIRST = "Mathematics" #@param ["Total Marks","Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
SECOND = "General Science" #@param ["Total Marks", "Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
THIRD = "English" #@param ["Total Marks", "Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}

import re, os
from bs4 import BeautifulSoup
from operator import itemgetter

selTable = {
    "align": "center",
    "width": "550",
    "border": "0",
    "cellpadding":"3",
    "cellspacing": "1",
    "class": "black12bold"
}
allRollDATA_P = []
allRollDATA_A = []
allRollDATA_F = []
for roll in range(START, END+1):
  sid = {}
  with open(f'dpe/{roll}.html') as f:
    read_data = f.read()
  soup = BeautifulSoup(read_data, 'html5lib').find("table", attrs=selTable).text
  # print(soup)
  # os.exit()
  sid['name'] = re.findall(r'\tনামঃ\s(.+)', soup)[0]
  sid["father's name"] = re.findall(r'\tপিতার\sনামঃ\s(.+)', soup)[0]
  sid["mother's name"] = re.findall(r'\tমাতার\sনামঃ\s(.+)', soup)[0]
  sid['roll'] = int(re.findall(r'Roll\sNumber\n\t{2}:\n\t{2}\d{1,3}', soup)[0].split('\n')[2])
  sid['student id'] = int(re.findall(r'Student\sID\n\t{2}:\n\t{2}\d{1,16}', soup)[0].split('\n')[2])
  sid['Year'] = int(re.findall(r'Year\n\t{2}:\n\t{2}\d{1,4}', soup)[0].split('\n')[2])
  sid['result'] = re.findall(r'Result\n\t{2}:\n\t{2}\w{1,6}', soup)[0].split('\n')[2][2:]
  if sid['result'] == 'Absent':
    allRollDATA_A.append(sid)
  elif sid['result'] == 'Failed':
    allRollDATA_F.append(sid)
  elif sid['result'] == 'Passed':
    sid['Bangla'] = int(re.findall(r'Bangla\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['English'] = int(re.findall(r'English\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['Mathematics'] = int(re.findall(r'Mathematics\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['Bangladesh and Global Studies'] = int(re.findall(r'Bangladesh\sand\sGlobal\sStudies\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['Religion'] = int(re.findall(r'Religion\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['General Science'] = int(re.findall(r'General\sScience\n\t{2}:\n\t{2}Grade:.+', soup)[0].split('\n')[2][-3:])
    sid['Total Marks'] = re.findall(r'GPA:\s\d.\d{2}\s\S\sTotal\sMarks:.+', soup)[0][-3:]
    allRollDATA_P.append(sid)
  
   
newlist = sorted(allRollDATA_P, key=itemgetter(Total_Marks, FIRST, SECOND, THIRD), reverse=True)
final_data = f'Total Student : {len(allRollDATA_P+allRollDATA_F+allRollDATA_A)},' \
            f' Absent : {len(allRollDATA_A)}' \
            f' Passed : {len(allRollDATA_P)}' \
            f' Failed : {len(allRollDATA_F)}\n\n'

def print_final(a):
  final_data = ""
  for id,std in enumerate(a):
    final_data += f'EST_Roll : {id+1}\n'
    for k,v in std.items():
      final_data += f'{k} : {v}\n'
    final_data += '\n'
  return final_data

for do in [newlist, allRollDATA_F, allRollDATA_A]:
  final_data += print_final(do)

with open(f'Sorted_Result_.txt', 'w') as f:
          f.write(final_data)

print("All Done")
print("Check file /content/Sorted_Result_.txt")
# print(newlist)

# result.dpe.gov.bd

In [0]:
#email biplobsd11@gmail.com
#@markdown <h3>⬅️ Click Here to START DOWNLOADING.</h3>
#@markdown <br><center><img src='http://dperesult.teletalk.com.bd/images/index__01.jpg' height="200" alt="PEC"/></center>
#@markdown <center><h3>PEC<br />Batch download marks sheet</h3></center><br>
#@markdown Student ID range  :
START =  1120193080500071#@param {type:"number"}
END =  1120193080500181#@param {type:"number"}

import requests, pathlib, os
from bs4 import BeautifulSoup
from IPython.display import HTML
from threading import Thread
headers = {
    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.46 Safari/537.36',
}

pathlib.Path('dpe2').mkdir(mode=0o777, exist_ok=True)
url = "http://202.51.191.190:8431/ResultSingleIDWise.aspx"

login_data = {
    "__EVENTTARGET": "",
    "__EVENTARGUMENT": "",
    "btnSubmit": "সমর্পন করুন",
}

def buildRange(START, END, numsplits):
    value = (END+1) - START
    lst = []
    for i in range(numsplits):
        if i == 0:
            lst.append([START, START + int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))])
        else:
            lst.append([START + int(round(1 + i * value/(numsplits*1.0),0)), START + int(round(1 + i * value/(numsplits*1.0) + value/(numsplits*1.0)-1, 0))])
    return lst

splitRange = buildRange(START, END, 2)
def downloader(START, END):
  with requests.Session() as s:
      for roll in range(START, END):
        if not os.path.isfile(f'dpe2/{roll}.html'):
          r = s.get(url, headers=headers)
          soup = BeautifulSoup(r.content, 'html5lib')
          login_data["__VIEWSTATE"] = soup.find("input", attrs={"name": "__VIEWSTATE"})["value"]
          login_data["__VIEWSTATEGENERATOR"] = soup.find("input", attrs={"name": "__VIEWSTATEGENERATOR"})["value"]
          login_data["__EVENTVALIDATION"] = soup.find("input", attrs={"name": "__EVENTVALIDATION"})["value"]
          login_data["txtStdID"] = roll
          for do in range(10):
            content = s.post(url, data=login_data, headers=headers).text
            try:
              if int(BeautifulSoup(content, 'html5lib').find("span", attrs={"id": "lblStudtId"}).text) == roll:
                with open(f'dpe2/{roll}.html', 'w') as f:
                    f.write(content)
                break
            except ValueError:
              print(f"id {roll} Retrying... {do}")
              continue
          print(f"Done {roll}")
        else:
          print(f"Done {roll}")
for ck in range(3):
  downloaders = [
          Thread(
              target=downloader, 
              args=(START, END),
          )
          for START,END in splitRange
          ]
  for th in downloaders:
      th.start()
  for th in downloaders:
      th.join()


In [0]:
#email biplobsd11@gmail.com
#@markdown <h3>⬅️ Click Here to START SORTING.</h3>
#@markdown <br><center><img src='http://dperesult.teletalk.com.bd/images/index__01.jpg' height="200" alt="PEC"/></center>
#@markdown <center><h3>PEC<br />Sort marks sheet</h3></center><br>
#@markdown Student ID range  :
START =  1120193080500071#@param {type:"number"}
END =  1120193080500181#@param {type:"number"}
#@markdown Subject priority :
Total_Marks = "Total Marks" #@param ["Total Marks","Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
FIRST = "Mathematics" #@param ["Total Marks","Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
SECOND = "General Science" #@param ["Total Marks", "Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}
THIRD = "English" #@param ["Total Marks", "Bangla", 'English', 'Mathematics', 'Bangladesh and Global Studies', 'Religion', 'General Science'] {type:"string"}

import re
from bs4 import BeautifulSoup
from operator import itemgetter

allRollDATA_P = []
allRollDATA_A = []
allRollDATA_F = []
for roll in range(START, END+1):
  sid = {}
  with open(f'dpe2/{roll}.html') as f:
    read_data = f.read()
  soup = BeautifulSoup(read_data, 'html5lib')
  sid['lbltha'] = soup.find("span", attrs={"id":"lbltha"}).text
  sid['lblSchName'] = soup.find("span", attrs={"id":"lblSchName"}).text
  sid['lblStudtName'] = soup.find("span", attrs={"id":"lblStudtName"}).text
  sid['lblFatherName'] = soup.find("span", attrs={"id":"lblFatherName"}).text
  sid['lblMotherName'] = soup.find("span", attrs={"id":"lblMotherName"}).text
  sid['roll'] = int(soup.find("span", attrs={"id":"lblStudtRoll"}).text)
  sid['student id'] = int(soup.find("span", attrs={"id":"lblStudtId"}).text)
  sid['result'] = soup.find("span", attrs={"id":"lblStatusTerm"}).text

  try:
    sid['Bangla'] = int(soup.find("span", attrs={"id":"lblBan0"}).text)
    sid['English'] = int(soup.find("span", attrs={"id":"lblEng0"}).text)
    sid['Mathematics'] = int(soup.find("span", attrs={"id":"lblMat0"}).text)
    sid['Bangladesh and Global Studies'] = int(soup.find("span", attrs={"id":"lblBvbp0"}).text)
    sid['Religion'] = int(soup.find("span", attrs={"id":"lblReg0"}).text)
    sid['General Science'] = int(soup.find("span", attrs={"id":"lblPSci0"}).text)
    sid['Total Marks'] = int(soup.find("span", attrs={"id":"lblTot"}).text)
  except:
    sid['Bangla'] = None
    sid['English'] = None
    sid['Mathematics'] = None
    sid['Bangladesh and Global Studies'] = None
    sid['Religion'] = None
    sid['General Science'] = None
    sid['Total Marks'] = 0
  
  if sid['result'] == 'অনুপস্থিত':
    allRollDATA_A.append(sid)
  elif sid['result'] == 'অনুত্তীর্ণ':
    allRollDATA_F.append(sid)
  elif sid['result'] == 'কৃতকার্য':
    allRollDATA_P.append(sid)
   
newlist = sorted(allRollDATA_P, key=itemgetter(Total_Marks, FIRST, SECOND, THIRD), reverse=True)
final_data = f'Total Student : {len(allRollDATA_P+allRollDATA_F+allRollDATA_A)},' \
            f' Absent : {len(allRollDATA_A)}' \
            f' Passed : {len(allRollDATA_P)}' \
            f' Failed : {len(allRollDATA_F)}\n\n'

def print_final(a):
  final_data = ""
  for id,std in enumerate(a):
    final_data += f'EST_Roll : {id+1}\n'
    for k,v in std.items():
      final_data += f'{k} : {v}\n'
    final_data += '\n'
  return final_data

for do in [newlist, allRollDATA_F, allRollDATA_A]:
  final_data += print_final(do)

with open(f'new_Sorted_Result.txt', 'w') as f:
          f.write(final_data)
          
print("All Done")
print("Check file /content/new_Sorted_Result.txt")